In [ ]:
# convert pytorch model to onnx format

%run export.py --weights ../weights/yolov7_lw.pt \
        --grid --simplify \
        --topk-all 100 --iou-thres 0.65 --conf-thres 0.35 \
        --img-size 640 640 --max-wh 640 

In [ ]:
# convert onnx format to tensorflow saved model 

import onnx
from onnx_tf.backend import prepare

onnx_model = onnx.load("weights/yolov7_lw.onnx") 
tf_rep = prepare(onnx_model)  
tf_rep.export_graph("weights/result") 

In [ ]:
#Weight Quantization 

import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('weights/yolov7_lw')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
with open('yolov7_lw_wq.tflite', 'wb') as w:
    w.write(tflite_quant_model)

In [ ]:
# Float16 Quantization 
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('weights/yolov7_lw')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()
with open('yolov7_lw_f16.tflite', 'wb') as w:
    w.write(tflite_quant_model)

In [ ]:
# generate data sets for calibration

from PIL import Image
import os, glob
import numpy as np

dataset = []

files = glob.glob("*.JPG")
for file in files:
    image = Image.open(file)
    image = image.convert("RGB")
    data = np.asarray(image)
    dataset.append(data)

dataset = np.array(dataset)
np.save("images/custom_dataset/images/train", dataset)


In [ ]:
# Integer Quantization 

import tensorflow as tf
import numpy as np

def representative_dataset_gen():
    for image in raw_test_data:
        image = tf.image.resize(image, (640, 640))
        image = image[np.newaxis,:,:,:]
        yield [image]

raw_test_data = np.load('train.npy', allow_pickle=True)

# Integer Quantization - Input/Output=float32
converter = tf.lite.TFLiteConverter.from_saved_model('weights/yolov7_lw')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
tflite_quant_model = converter.convert()
with open('yolov7_lw_iq.tflite', 'wb') as w:
    w.write(tflite_quant_model)

In [ ]:
# Full Integer Quantization 

import tensorflow as tf
import numpy as np

def representative_dataset_gen():
    for image in raw_test_data:
        image = tf.image.resize(image, (640, 640))
        image = image[np.newaxis,:,:,:]
        yield [image]

raw_test_data = np.load('train.npy', allow_pickle=True)

# Full Integer Quantization - Input/Output=float32
converter = tf.lite.TFLiteConverter.from_saved_model('weights/yolov7_lw')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
converter.representative_dataset = representative_dataset_gen
tflite_quant_model = converter.convert()
with open('yolov7_lw_fiq.tflite', 'wb') as w:
    w.write(tflite_quant_model)